# Expected Completion

In [1]:
import pandas as pd
import numpy as np

In [2]:
plays = pd.read_csv("data/pass_receiver.csv")

read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [4]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [6]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [7]:
tracking

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978944,2021110100,4433,NaN,54,2021-11-02T03:20:26.500,NaN,football,right,24.17,27.89,2.47,2.28,0.25,NaN,NaN,None
978945,2021110100,4433,NaN,55,2021-11-02T03:20:26.600,NaN,football,right,24.36,28.03,2.36,2.16,0.24,NaN,NaN,None
978946,2021110100,4433,NaN,56,2021-11-02T03:20:26.700,NaN,football,right,24.55,28.17,2.25,1.45,0.23,NaN,NaN,None
978947,2021110100,4433,NaN,57,2021-11-02T03:20:26.800,NaN,football,right,24.73,28.31,2.28,0.72,0.23,NaN,NaN,None
